# 5. Value Betting Calculator

In this notebook we'll make use of our match dataset and machine learning model we've built in the last stage, to predict the chances for each player to win in any given match.

We'll then compare these chances with the odds available for those matches, and build a Dataframe where each row will contain information regarding which player we would have to bet on!

Let's import the libraries needed for this notebook:

In [1]:
import pandas as pd
from math import exp
import pickle

Now, we'll load the model trained in last notebook:

In [2]:
model = pickle.load(open("logistic_regression.lr", 'rb'))

Now, let's open the Coefficients dataframe saved in the last notebook, and let's create a dictionary object based on the dataframe, where the key is going to be the attribute name and the value will be the attribute's coefficient:

In [3]:
coefs_df = pd.read_csv("csv/Coefficients.csv")

In [4]:
coefs_df.head()

,Column,Coef
0,Indoor,-0.008205
1,Outdoor,0.018288
2,Carpet,0.018235
3,Clay,-0.003667
4,Grass,-0.030138


In [5]:
coefs_map = {}
for ix, row in coefs_df.iterrows():

    attribute = row['Column']
    attribute_coef = row['Coef']

    coefs_map[attribute] = attribute_coef

In [6]:
coefs_map

{'Indoor': -0.008205490055114922,
 'Outdoor': 0.018288190776828448,
 'Carpet': 0.018235098719900585,
 'Clay': -0.0036670657947837455,
 'Grass': -0.030137893520682982,
 'Hard': 0.02565256131728023,
 'ATP250': 0.010034866580374424,
 'ATP500': 0.017153517648049232,
 'Grand Slam': -0.014697553313709538,
 'Masters 1000': 0.03930679973658772,
 'Masters Cup': -0.04171492992958769,
 '1st Round': -0.007050532204758627,
 '2nd Round': 0.0160313689626121,
 '3rd Round': -0.07591470921881736,
 '4th Round': 0.019890644865569376,
 'Quarterfinals': -0.03044797716182537,
 'Round Robin': 0.08453897012931598,
 'Semifinals': -0.012502992714257024,
 'The Final': 0.01553792806387606,
 'Rank Index': 0.3627017152187937,
 'Pl0 Recent Form': -0.27294250215186305,
 'Pl0 Form': 0.2708226143231703,
 'Pl1 Recent Form': 0.3138160892166305,
 'Pl1 Form': -0.5232964971778234,
 'Pl0 Perf. vs Similar Opponent': -2.4604492270974863,
 'Pl1 Perf. vs Similar Opponent': 2.234436723553626,
 'Pl0 Surface Performance': -3.0215764

Let's make a quick analysis on the greatest indicators for player 1 to win a match against player 0.
Let's see which attributes matter the most:

In [7]:
coefs_df[abs(coefs_df['Coef']) > 0.1].sort_values('Coef', ascending = False)

,Column,Coef
27,Pl1 Surface Performance,3.389839
25,Pl1 Perf. vs Similar Opponent,2.234437
19,Rank Index,0.362702
22,Pl1 Recent Form,0.313816
21,Pl0 Form,0.270823
20,Pl0 Recent Form,-0.272943
23,Pl1 Form,-0.523296
24,Pl0 Perf. vs Similar Opponent,-2.460449
26,Pl0 Surface Performance,-3.021576


We can see that the most important features are:
- Player performance on specific surface
- Player performance against similar ranked opponents
- The opponent's form
- The rank index
- The players recent form

Let's load our matches Dataset

In [8]:
df = pd.read_csv("csv/FeatureCalculated_Data.csv")

Now our goal is to build another Dataframe, containing the informations about the the players, who wins each match, the average and maximum odds available for each player, and we'll use our loaded model to calculate the predicted chances to win for each player:

In [9]:
betting_data_df = df[(df["Avg_Pl0"] > 1) & (df["Avg_Pl1"] > 1) & (df["Max_Pl0"] > 1) & (df["Max_Pl1"] > 1)]

In [10]:
inputs = betting_data_df.iloc[:, 10:]

In [11]:
predicted_outputs = model.predict_proba(inputs)

In [12]:
index = 0
for ix, row in betting_data_df.iterrows():
    prob_pl0 = predicted_outputs[index][0] 
    prob_pl1 = predicted_outputs[index][1] 
    betting_data_df.at[ix, 'Pl0 %'] = prob_pl0
    betting_data_df.at[ix, 'Pl1 %'] = prob_pl1
    index += 1

C:\Users\acorsett\python_installation\envs\py3-TF2.0\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\acorsett\python_installation\envs\py3-TF2.0\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
betting_data_df.head()

,Avg_Pl1,Avg_Pl0,Date,Pl1_Rank,Player 1,Max_Pl1,Max_Pl0,Pl0_Rank,Player 0,Won,...,Pl0 Recent Form,Pl0 Form,Pl1 Recent Form,Pl1 Form,Pl0 Perf. vs Similar Opponent,Pl1 Perf. vs Similar Opponent,Pl0 Surface Performance,Pl1 Surface Performance,Pl0 %,Pl1 %
29521,2.46,1.52,2010-04-19,58,Falla A.,2.79,1.62,67,De Bakker T.,0.0,...,0.57,0.50,0.43,0.40,0.50,0.39,0.46,0.39,0.619670,0.380330
29522,3.12,1.34,2010-04-19,86,Hajek J.,3.64,1.40,61,Starace P.,1.0,...,0.43,0.35,0.14,0.30,0.52,0.40,0.52,0.39,0.671717,0.328283
29523,1.77,1.99,2010-04-19,74,Fognini F.,1.85,2.14,59,Schwank E.,0.0,...,0.43,0.35,0.14,0.30,0.46,0.60,0.45,0.58,0.326398,0.673602
29524,1.36,3.02,2010-04-19,42,Garcia-Lopez G.,1.45,3.25,122,Rochus C.,1.0,...,0.00,0.10,0.43,0.35,0.36,0.59,0.43,0.50,0.325714,0.674286
29525,2.63,1.46,2010-04-19,64,Nieminen J.,3.09,1.50,33,Bellucci T.,0.0,...,0.43,0.65,0.57,0.45,0.57,0.46,0.53,0.53,0.521146,0.478854


Let's define a helper funxtion that takes a change in input and returns the 'fair' odds for that chance value.

In [14]:
def right_odds(chance):
    return round(1/chance,2)

Now, we have everything we need to make decisions based on which player, for each match, we get the most value betting on!
In fact, next we're going to append to this betting dataframe another 4 columns.

The fair odds for both player 0 and player 1, based on the chances of each player to win according to the model, and the value that we would get betting on each player at the max odds available!

In [15]:
for ix, row in betting_data_df.iterrows():
    right_odds_p0 = right_odds(row['Pl0 %'])
    right_odds_p1 = right_odds(row['Pl1 %'])
    bet_value_p0 = (row['Max_Pl0'] - right_odds_p0)/max(right_odds_p0,row['Max_Pl0'])
    bet_value_p1 = (row['Max_Pl1'] - right_odds_p1)/max(right_odds_p1,row['Max_Pl1'])
    betting_data_df.at[ix, 'Fair odds for Pl0'] = right_odds_p0
    betting_data_df.at[ix, 'Fair odds for Pl1'] = right_odds_p1
    betting_data_df.at[ix, 'Bet on Pl0 Value'] = bet_value_p0
    betting_data_df.at[ix, 'Bet on Pl1 Value'] = bet_value_p1

C:\Users\acorsett\python_installation\envs\py3-TF2.0\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\acorsett\python_installation\envs\py3-TF2.0\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [18]:
betting_data_df.head()

,Avg_Pl1,Avg_Pl0,Date,Pl1_Rank,Player 1,Max_Pl1,Max_Pl0,Pl0_Rank,Player 0,Won,...,Pl0 Perf. vs Similar Opponent,Pl1 Perf. vs Similar Opponent,Pl0 Surface Performance,Pl1 Surface Performance,Pl0 %,Pl1 %,Fair odds for Pl0,Fair odds for Pl1,Bet on Pl0 Value,Bet on Pl1 Value
29521,2.46,1.52,2010-04-19,58,Falla A.,2.79,1.62,67,De Bakker T.,0.0,...,0.50,0.39,0.46,0.39,0.619670,0.380330,1.61,2.63,0.006173,0.057348
29522,3.12,1.34,2010-04-19,86,Hajek J.,3.64,1.40,61,Starace P.,1.0,...,0.52,0.40,0.52,0.39,0.671717,0.328283,1.49,3.05,-0.060403,0.162088
29523,1.77,1.99,2010-04-19,74,Fognini F.,1.85,2.14,59,Schwank E.,0.0,...,0.46,0.60,0.45,0.58,0.326398,0.673602,3.06,1.48,-0.300654,0.200000
29524,1.36,3.02,2010-04-19,42,Garcia-Lopez G.,1.45,3.25,122,Rochus C.,1.0,...,0.36,0.59,0.43,0.50,0.325714,0.674286,3.07,1.48,0.055385,-0.020270
29525,2.63,1.46,2010-04-19,64,Nieminen J.,3.09,1.50,33,Bellucci T.,0.0,...,0.57,0.46,0.53,0.53,0.521146,0.478854,1.92,2.09,-0.218750,0.323625


In [20]:
betting_data_df.rename(columns={"Avg_Pl1":"Pl1 Avg odds", "Avg_Pl0":"Pl0 Avg odds", "Max_Pl1":"Pl1 Max odds", 
                                "Max_Pl0":"Pl0 Max odds", "Won":"Winner"},inplace = True)

C:\Users\acorsett\python_installation\envs\py3-TF2.0\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


We just obtained 17104 matches where we can bet on the player who's value is higher!
Let's save this dataframe!

In [21]:
betting_data_df.to_csv("csv/Betting_Data.csv", index=False)